## Demo notebook to check out the Google Places API and the results

In [1]:
import requests
from openai import OpenAI
import os

### Define functions

In [2]:
#get nearby restaurants
def get_nearby_restaurants(api_key, location, radius=200, keyword='restaurant', num_results=5):
    '''
    Returns the restaurants from the API call based on location, radius
    '''
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    
    params = {
        'location': location,
        'radius': radius,
        'keyword': keyword,
        'key': api_key
    }
    
    response = requests.get(base_url, params=params)
    results = response.json().get('results', [])

    # Limit the number of results to the top 'num_results'
    results = results[:num_results]
    
    return results


def get_place_details(api_key, place_id):
    '''
    Returns details on a place defined by the place_id, e.g. reviews
    '''
    base_url = 'https://maps.googleapis.com/maps/api/place/details/json'

    params = {
        'place_id': place_id,
        'key': api_key
    }

    response = requests.get(base_url, params=params)
    result = response.json().get('result', {})

    return result


def generate_summary(client, reviews):
    #Concatenate all reviews
    review_text = "\n".join(review.get('text', '') for review in reviews)

    #Define the prompt for GPT
    prompt = f"Summarize the reviews for this place:\n{review_text}"

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt},
        ]
    )
    

    #Extract the generated summary from GPT's response
    summary = response.choices[0].message.content

    return summary
    

### Set api keys from env

In [3]:
#get api keys from env file
google_api_key = os.environ.get("GOOGLE_API_KEY")
openai_api_key = os.environ.get("OPENAI_API_KEY")
if google_api_key is not None:
    print("Google key found!")
if openai_api_key is not None:
    print("OpenAI key found!")

Google key found!
OpenAI key found!


### Search for places using Google Places API

In [7]:
# Munich center coordinates (for example)
munich_center = '48.1363964,11.5609501'

# Set radius and keyword
restaurants = get_nearby_restaurants(google_api_key, munich_center, radius=200, keyword='restaurant', num_results=3)

# Print the names of the found restaurants
for restaurant in restaurants:
    #Print Restaurant name and average rating
    print("Restaurant name: ", restaurant.get('name'))
    print("Rating:  ", restaurant.get('rating'))

Restaurant name:  La Vecchia Masseria
Rating:   4
Restaurant name:  Münchner Stubn
Rating:   4.3
Restaurant name:  Hindukush
Rating:   4.4


### Generate Summary with GPT3.5

In [6]:
from openai import OpenAI


#Client, automatically searches for OPENAI_API_KEY in environment and sets as key
client = OpenAI()


#Iterate over restaurants: We want to generate a review for each one
for restaurant in restaurants:
    #Get place details 
    restaurant_id = restaurant.get('place_id')
    place_details = get_place_details(google_api_key, restaurant_id)

    #Get reviews for the place
    reviews = place_details.get('reviews', [])
    
    #Print Restaurant name
    print("Restaurant name: ", restaurant.get('name'))

    #Generate the summary
    summary = generate_summary(client, reviews)
    print(f"Summary: {summary}\n")

Restaurant name:  La Vecchia Masseria
Summary: Overall, the reviews for this place are positive. The ambiance is described as nice and the atmosphere is great. The food is soulful and amazing, with classic dishes like Bolognese and Gorgonzola being perfect. The prices are fair and not considered a rip-off for tourists. The outside seating is enjoyable and offers a nice atmosphere. The service is efficient and responsive, although not overly warm nor super attentive. One reviewer mentions that the restaurant has been a favorite for many years and maintains consistent quality.

Restaurant name:  Münchner Stubn
Summary: Overall, the reviews for this place are positive. Many customers enjoyed the local specialties on the menu, the variety of beers, and the decent flavor and presentation of the food. The portion sizes were also praised as being generous. The servers were responsive and professional, and the menu was available in both German and English. Some customers mentioned feeling welc

## Prompt trials

In [13]:
def generate_summary_prompt(client, prompt):

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt},
        ]
    )
    

    #Extract the generated summary from GPT's response
    summary = response.choices[0].message.content

    return summary

In [14]:
#Concatenate all reviews


"""
prompt1 = f"Summarize the reviews for this place as short and easy to understand as possible:\n{review_text}"
prompt2 = f"Summarize the reviews for this restaurant please. For this, please provide positive and negative aspects with bulletpoints. Positive ones should start with 'Pros:' and negative ones with 'Cons: ', These are the reviews:\n{review_text}"
prompt3 = f"Can you give me the most common positive and negative aspects in the reviews of this restaurant:\n{review_text}"
# prompt4 = f"Summarize the reviews for this place:\n{review_text}"
"""

print("PROMPT 1: ")
for restaurant in restaurants:
    #Get place details 
    restaurant_id = restaurant.get('place_id')
    place_details = get_place_details(google_api_key, restaurant_id)

    #Get reviews for the place
    reviews = place_details.get('reviews', [])
    review_text = "\n".join(review.get('text', '') for review in reviews)
    print("review_text")
    prompt1 = f"Look through these reviews and summarize all of them in 2 to 3 sentences with the information and experiences that might be the most interesting to potential customers:\n{review_text}"
    
    #Print Restaurant name
    print("Restaurant name: ", restaurant.get('name'))

    #Generate the summary
    summary = generate_summary_prompt(client, prompt1)
    print(f"Summary: {summary}\n")





PROMPT 1: 
review_text
Restaurant name:  La Vecchia Masseria
Summary: This restaurant is described as a hidden gem with a lovely atmosphere and amazing soulful food. Prices are reasonable and the outside seating area is praised. Some customers mention enjoying the Christmas decorations while others mention the consistent quality of the food over the years. The service is described as efficient.

review_text
Restaurant name:  Münchner Stubn
Summary: This restaurant is busy but has responsive and professional servers. They have a variety of local specialties and a great variety of beers on tap. The food is decent, well-presented, and comes in generous portions. They have a German and English menu and the staff is friendly and helpful. The vibe is fun with live music during Oktoberfest. The only negative is that one waiter was rude and rushed, but the overall dining experience is great. It's a cozy, moderately priced restaurant with efficient service and delicious food. It's conveniently 

In [16]:
print("PROMPT 2: ")
for restaurant in restaurants:
    #Get place details 
    restaurant_id = restaurant.get('place_id')
    place_details = get_place_details(google_api_key, restaurant_id)

    #Get reviews for the place
    reviews = place_details.get('reviews', [])
    review_text = "\n".join(review.get('text', '') for review in reviews)
    prompt2 = f"Look through these reviews and provide a summary in form of bulletpoints about the most common opinions:\n{review_text}"
    
    #Print Restaurant name
    print("Restaurant name: ", restaurant.get('name'))

    #Generate the summary
    summary = generate_summary_prompt(client, prompt2)
    print(f"Summary: {summary}\n")


PROMPT 2: 
Restaurant name:  La Vecchia Masseria
Summary: - The restaurant has a lovely and quaint atmosphere, with a bustling atmosphere due to families.
- The location is considered a hidden gem, close to the Aldstadt but a bit of a walk away.
- The food is soulful and amazing, with classics like Bolognese and Gorgonzola being perfect choices.
- Prices have increased recently but are still typical for a similar restaurant.
- It is recommended to make a reservation, although going early may allow one to skip it.
- All pizzas are €8.99 on Sundays.
- The ambience is nice, but it can be a bit loud when the restaurant is full.
- The Christmas decorations add to the enjoyable atmosphere.
- The prices are fair and not considered a rip-off for tourists.
- The overall quality of food is great, with good service.
- The outside seating in a quiet street is very nice, with a good vibe and service.
- Basic dishes have good quality, and the reviewer would return for beer and the atmosphere.
- The 

In [19]:
print("PROMPT 3: ")
for restaurant in restaurants:
    #Get place details 
    restaurant_id = restaurant.get('place_id')
    place_details = get_place_details(google_api_key, restaurant_id)

    #Get reviews for the place
    reviews = place_details.get('reviews', [])
    review_text = "\n".join(review.get('text', '') for review in reviews)
    prompt = f"Look through these reviews and provide a 3 sentence summary, highlighting the following topics: food, ambience, service, price, consistency, creativeness, size. Please also provide one meal/option that seems the most popular, if there is one. These are the reviews:  \n{review_text}"
    
    #Print Restaurant name
    print("Restaurant name: ", restaurant.get('name'))

    #Generate the summary
    summary = generate_summary_prompt(client, prompt)
    print(f"Summary: {summary}\n")

PROMPT 3: 
Restaurant name:  La Vecchia Masseria
Summary: Summary:

- Food: The food is described as soulful, amazing, and good quality. Pasta dishes like Bolognese and Gorgonzola are particularly praised.
- Ambience: The ambience is nice, with a great atmosphere and Christmas decorations. Outside seating on a quiet street is also enjoyed.
- Service: The service is efficient and responsive, but not overly warm or attentive.
- Price: Prices are fair and typical for a similar restaurant, with recent increases noted.
- Consistency: One reviewer mentions that the restaurant has maintained the same quality and workers for several years.
- Creativeness: No specific comments about the creativity of the food.
- Size: No specific comments about the portion sizes.

Most popular meal/option: All pizzas are €8.99 on Sundays.

Restaurant name:  Münchner Stubn
Summary: Summary:
- The food at this German restaurant is decent in flavor with generous portions and great presentation. They offer a variet

## Notes on experimentation

- text or bullet points?
- how long should a summary be, e.g. 3-4 sentences, 6-10 bullet points?
- repetitive sentences
- highlighting of specific criteria, e.g. food, ambience, price, service
- popular meal option?